In [ ]:
# !pip install kfp==1.8.18

In [27]:
import kfp
import kfp.dsl as dsl
from kfp import components
import kubernetes as k8s
from kfp import compiler, dsl
from kubernetes.client import V1VolumeMount
import pprint
import numpy as np


@dsl.pipeline(
    name="Zone delineation pipeline",
    description="pipeline for soil sampling and delineation"
)
def full_pipeline(size: str="2Gi"):
    
    vop = dsl.PipelineVolume(
        pvc="biosens-agrovision-data-volume",
        name="biosens-agrovision-data-volume",
    )

    download_component = kfp.components.load_component_from_file('download_model.yaml')
    zoning_model = kfp.components.load_component_from_file('zoning_model.yaml')
    postprocessing_model = kfp.components.load_component_from_file('postprocessing_model.yaml')
    
    download_component_task = download_component(
        "biosens-test-inference-service",
        "/mnt/InferenceData/"
        ).add_pvolumes({"/mnt": vop})
    
    
    zoning_model_task = zoning_model(
        4,
        10.0,
        "/mnt/InferenceData/DataTest/shp/TestParcel_Belanovica1.shp",
        "/mnt/InferenceData/DataTest/GeoTiffs/Belanovica1_processed_transparent_mosaic_red.tif",
        "/mnt/InferenceData/DataTest/GeoTiffs/Belanovica1_processed_transparent_mosaic_green.tif",
        "/mnt/InferenceData/DataTest/GeoTiffs/Belanovica1_processed_transparent_mosaic_blue.tif",
        "/mnt/InferenceData/DataTest/GeoTiffs/Belanovica1_processed_transparent_mosaic_nir.tif",
        "/mnt/InferenceData/DataTest/GeoTiffs/Belanovica1_processed_transparent_mosaic_red edge.tif",
        "/mnt/InferenceData/DataTest/zoning_results/run1/",
        download_component_task.output).add_pvolumes({"/mnt": vop})
    
    
    
    postprocessing_model = postprocessing_model(
    "blueberry-results",
    "/mnt/InferenceData/DataTest/zoning_results/run1/",
    "minio-push1/",
    zoning_model_task.output
    ).add_pvolumes({"/mnt": vop})



    print("gotovo sve")
if __name__ == '__main__':
    file_name = "full_pipeline.yaml"
    kfp.compiler.Compiler().compile(full_pipeline, file_name)

gotovo sve


In [28]:
import kfp
import random
client=kfp.Client()
filepath = '/home/jovyan/Zoning/' + file_name
name = 'ZoningPipeline12'
print("Uploaded pipeline:", name)
pipeline = client.pipeline_uploads.upload_pipeline(filepath, name=name)

Uploaded pipeline: ZoningPipeline12
